In [1]:
import sys
sys.path.append("../src")

from renderer import Renderer
from utils import get_experiment_name, save_cfg, set_seeds
from model_utils import get_grad_texture
from visualization import save_images, save_gif

import functools
import itertools

import cv2
import torch
from tqdm import tqdm

from score_sde.losses import get_optimizer
from score_sde.models.ema import ExponentialMovingAverage
from score_sde.utils import restore_checkpoint
from score_sde.models import utils as mutils
from score_sde.models import ncsnpp
from score_sde.sde_lib import VESDE
from score_sde.configs.ve import ffhq_256_ncsnpp_continuous as configs

cfg = {
    "seed": 0,
    "batch_size": 1,
    "image_size": 256,
    "texture_size": 2048,
    "saved_image_size": 256,
    "num_channels": 3,
    "image_save_dir": "../results",
    "device": torch.device("cuda:0"),
    "num_corrector_steps": 6,
    "sde_N": 1000,
    "snr": 0.02, # 0.035 large
    "num_optimization_steps_for_filled_texture": 350, # 400 large
    # "initial_noise_coefficient": 50.0
}
cfg["experiment_name"] = get_experiment_name()
# cfg["texture_path"] = "/local/home/aarslan/DECA/TestSamples/examples/results/33673/33673.png"
# cfg["obj_path"] = "/local/home/aarslan/DECA/TestSamples/examples/results/33673/33673.obj"
cfg["texture_path"] = "/local/home/aarslan/score-face/assets/FFHQ_results_2048_2020/33673.png"
cfg["obj_path"] = "/local/home/aarslan/score-face/assets/FFHQ_results_2048_2020/33673.obj"

set_seeds(cfg)
save_cfg(cfg)

ckpt_filename = "../assets/checkpoint_48.pth"
config = configs.get_config()
sde = VESDE(sigma_min=config.model.sigma_min, sigma_max=config.model.sigma_max, N=cfg["sde_N"])
sampling_eps = 1e-5

batch_size = cfg["batch_size"]
config.training.batch_size = batch_size
config.eval.batch_size = batch_size
config.device = cfg["device"]

sigmas = mutils.get_sigmas(config)
score_model = mutils.create_model(config)
score_model.eval()

optimizer = get_optimizer(config, score_model.parameters())
ema = ExponentialMovingAverage(score_model.parameters(),
                               decay=config.model.ema_rate)
state = dict(step=0, optimizer=optimizer, model=score_model, ema=ema)
state = restore_checkpoint(ckpt_filename, state, config.device)
ema.copy_to(score_model.parameters())

def render(texture, background, pixel_uvs, background_mask):
    texture_sampled = torch.nn.functional.grid_sample(torch.flip(texture, dims=[-2]), pixel_uvs, mode='bilinear', padding_mode='border', align_corners=False)
    return torch.where(background_mask, background, texture_sampled)

image_shape = (cfg["batch_size"], cfg["num_channels"], cfg["image_size"], cfg["image_size"])
texture_shape = (cfg["batch_size"], cfg["num_channels"], cfg["texture_size"], cfg["texture_size"])

def get_score_fn(sde, score_model):
    def score_fn(x, vec_t):
        labels = sde.marginal_prob(torch.zeros_like(x), vec_t)[1]
        score = score_model(x, labels)
        return score
    return score_fn

probability_flow = False
score_fn = get_score_fn(sde, score_model)
rsde = sde.reverse(score_fn=score_fn, probability_flow=probability_flow)
timesteps = torch.linspace(sde.T, sampling_eps, sde.N)
zeros = torch.zeros(size=image_shape, device=cfg["device"])

flame_texture = cv2.imread(cfg["texture_path"])[:, :, [2, 1, 0]]
flame_texture = cv2.resize(flame_texture, dsize=(cfg["texture_size"], cfg["texture_size"]))
flame_texture = torch.tensor(flame_texture, device=cfg["device"]).permute(2, 0, 1).unsqueeze(0) / 255.0  # (N, C, H, W)
filled_mask = flame_texture.sum(axis=1).bool().repeat(repeats=[1, 3, 1, 1])

prerender_results = {}
elevs = [0.0]
azimuths = [-35.0]
renderer = Renderer(cfg=cfg)

for elev, azimuth in tqdm(list(itertools.product(elevs, azimuths))):
    pixel_uvs, background_mask = renderer.render(texture=flame_texture, background=zeros, elev=elev, azimuth=azimuth, result_keys=["pixel_uvs", "background_mask"])
    prerender_results[f"{elev}_{azimuth}"] = {"pixel_uvs": pixel_uvs,
                                              "background_mask": background_mask}

pixel_uvs = torch.tensor(prerender_results["0.0_-35.0"]["pixel_uvs"], device=cfg["device"])
background_mask = torch.tensor(prerender_results["0.0_-35.0"]["background_mask"], device=cfg["device"]).bool()

demo_face = render(texture=flame_texture, background=zeros, pixel_uvs=pixel_uvs, background_mask=background_mask)
save_images(cfg=cfg, _images=demo_face, image_type="demo_face", iteration=0)

with torch.no_grad():
    for i in tqdm(list(range(sde.N))):
        t = timesteps[i]
        vec_t = torch.ones(cfg["batch_size"], device=cfg["device"]) * t
        alpha = torch.ones_like(vec_t)
        
        background_mean, std = sde.marginal_prob(zeros, vec_t)
        background = background_mean + torch.randn(size=image_shape, device=cfg["device"]) * std[:, None, None, None]

        random_texture = torch.randn(size=texture_shape, device=cfg["device"]) * std[:, None, None, None] # * cfg["initial_noise_coefficient"] / 348.0

        if i == 0:    
            unfilled_texture = random_texture * (1 - 1 * filled_mask)
        
        if i <= sde.N - cfg["num_optimization_steps_for_filled_texture"]:
            filled_texture = (flame_texture + random_texture) * (1 * filled_mask)

        for j in range(cfg["num_corrector_steps"]):
            texture = torch.where(filled_mask, filled_texture, unfilled_texture)
            face = render(texture=texture, background=background, pixel_uvs=pixel_uvs, background_mask=background_mask)
            grad_face = score_fn(face, vec_t)
            render_func = functools.partial(render, background=background, pixel_uvs=pixel_uvs, background_mask=background_mask)
            grad_texture = get_grad_texture(texture=texture, grad_face=grad_face, render_func=render_func)
            grad_filled_texture = grad_texture * filled_mask
            grad_unfilled_texture = grad_texture * (1 - 1 * filled_mask)

            grad_texture_norm = torch.norm(grad_texture.reshape(grad_texture.shape[0], -1), dim=-1).mean()

            if j == cfg["num_corrector_steps"] - 1:
                grad_texture_sum = grad_texture.sum(axis=1)
                grad_texture_zero_one = 1 * torch.logical_and(grad_texture_sum, torch.ones_like(grad_texture_sum)).unsqueeze(0).repeat(repeats=[1, 3, 1, 1])

            noise_texture = torch.randn_like(texture)
            noise_texture_norm = torch.norm(noise_texture.reshape(noise_texture.shape[0], -1), dim=-1).mean()
            step_size_texture = (cfg["snr"] * noise_texture_norm / grad_texture_norm) ** 2 * 2 * alpha

            if i >= sde.N - cfg["num_optimization_steps_for_filled_texture"]:
                filled_texture_mean = filled_texture + step_size_texture[:, None, None, None] * grad_filled_texture
                filled_texture = filled_texture_mean + torch.sqrt(step_size_texture * 2)[:, None, None, None] * noise_texture
            
            unfilled_texture_mean = unfilled_texture + step_size_texture[:, None, None, None] * grad_unfilled_texture
            unfilled_texture = unfilled_texture_mean + torch.sqrt(step_size_texture * 2)[:, None, None, None] * noise_texture

        if i == 0:
            save_images(cfg, _images=grad_texture_zero_one, image_type="texture_gradient", iteration=i)
            save_images(cfg, _images=(1 * background_mask), image_type="background_mask", iteration=i)
        if i % 10 == 0:
            save_images(cfg=cfg, _images=face, image_type="face", iteration=i)
            save_images(cfg=cfg, _images=texture, image_type="texture", iteration=i)
            save_images(cfg=cfg, _images=background, image_type="background", iteration=i)
        if i == sde.N - 1:
            save_gif(cfg=cfg)


/local/home/aarslan/miniconda3/envs/score-face/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-07-16 07:36:21.052860: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/local/home/aarslan/miniconda3/envs/score-face/lib/python3.9/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/local/home/aarslan/miniconda3/envs/score-face/lib/python3.9/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt
/local/home/aarslan/miniconda3/envs/score-face/lib/python3.9/site-packages/pytorch3d/io/utils.py:66: Use

In [1]:
import sys
sys.path.append("../src")
import itertools
from renderer import Renderer
from utils import get_experiment_name, save_cfg, set_seeds
from model_utils import get_grad_texture, get_grad_texture_and_background
from visualization import save_images, save_gif
import torch
import functools
import cv2
from tqdm import tqdm
from score_sde.losses import get_optimizer
from score_sde.models.ema import ExponentialMovingAverage
from score_sde.utils import restore_checkpoint
from score_sde.models import utils as mutils
from score_sde.models import ncsnpp
from score_sde.sde_lib import VESDE
from score_sde.configs.ve import ffhq_256_ncsnpp_continuous as configs


cfg = {
    "seed": 0,
    "batch_size": 1,
    "image_size": 256,
    "texture_size": 2048,
    "saved_image_size": 256,
    "num_channels": 3,
    "image_save_dir": "../results",
    "device": torch.device("cuda:0"),
    "num_corrector_steps": 6,
    "sde_N": 1000,
    "snr": 0.015, # tune
    "langevin_noise_coefficient": 1,
    "initial_noise_coefficient": 50.0,
    "num_optimization_steps_for_filled_part": 1000
}
cfg["obj_path"] = f"../assets/FFHQ_results_{cfg['texture_size']}_2020/40044.obj"
cfg["texture_path"] = f"../assets/FFHQ_results_{cfg['texture_size']}_2020/40044.png"
cfg["experiment_name"] = get_experiment_name()
set_seeds(cfg)
save_cfg(cfg)

ckpt_filename = "../assets/checkpoint_48.pth"
config = configs.get_config()
sde = VESDE(sigma_min=config.model.sigma_min, sigma_max=config.model.sigma_max, N=cfg["sde_N"])
sampling_eps = 1e-5

batch_size = cfg["batch_size"]
config.training.batch_size = batch_size
config.eval.batch_size = batch_size
config.device = cfg["device"]

sigmas = mutils.get_sigmas(config)
score_model = mutils.create_model(config)
score_model.eval()

optimizer = get_optimizer(config, score_model.parameters())
ema = ExponentialMovingAverage(score_model.parameters(),
                               decay=config.model.ema_rate)
state = dict(step=0, optimizer=optimizer, model=score_model, ema=ema)
state = restore_checkpoint(ckpt_filename, state, config.device)
ema.copy_to(score_model.parameters())

def render(texture, background, pixel_uvs, background_mask):
    texture_sampled = torch.nn.functional.grid_sample(torch.flip(texture, dims=[-2]), pixel_uvs, mode='bilinear', padding_mode='border', align_corners=False)
    return torch.where(background_mask, background, texture_sampled)

image_shape = (cfg["batch_size"], cfg["num_channels"], cfg["image_size"], cfg["image_size"])
texture_shape = (cfg["batch_size"], cfg["num_channels"], cfg["texture_size"], cfg["texture_size"])

prerender_results = {}
elevs = [0.0]
azimuths = [0.0]
renderer = Renderer(cfg=cfg)

background = torch.zeros(size=image_shape, device=cfg["device"])

# texture which was filled while azimuth=0
initial_texture = torch.tensor(cv2.imread(cfg["texture_path"])[:, :, [2, 1, 0]], device=cfg["device"]).permute(2, 0, 1).unsqueeze(0) / 255.0  # (N, C, H, W)

for elev, azimuth in tqdm(list(itertools.product(elevs, azimuths))):
    pixel_uvs, background_mask = renderer.render(texture=initial_texture, background=background, elev=elev, azimuth=azimuth, result_keys=["pixel_uvs", "background_mask"])
    prerender_results[f"{elev}_{azimuth}"] = {"pixel_uvs": pixel_uvs,
                                              "background_mask": background_mask}

pixel_uvs = torch.tensor(prerender_results["0.0_0.0"]["pixel_uvs"], device=cfg["device"])
background_mask = torch.tensor(prerender_results["0.0_0.0"]["background_mask"], device=cfg["device"]).bool()

initial_texture = torch.tensor(cv2.imread(cfg["texture_path"])[:, :, [2, 1, 0]], device=cfg["device"]).permute(2, 0, 1).unsqueeze(0) / 255.0  # (N, C, H, W)
filled_mask = initial_texture.sum(axis=1, keepdim=True).bool().repeat(repeats=[1, 3, 1, 1])
texture_half_size = (cfg["batch_size"], cfg["num_channels"], cfg["texture_size"], int(cfg["texture_size"] / 2))
filled_mask = torch.logical_and(filled_mask, torch.concat(tensors=(torch.ones(size=texture_half_size, device=cfg["device"]), torch.zeros(size=texture_half_size, device=cfg["device"])), dim=-1))
initial_texture_filled = initial_texture * filled_mask
# current_texture_filled = deepcopy(initial_texture_filled)
# initial_texture_unfilled = torch.randn_like(initial_texture) * (1 - 1 * filled_mask) * cfg["initial_noise_coefficient"]
# current_texture_unfilled = deepcopy(initial_texture_unfilled)
# current_texture = torch.where(filled_mask, current_texture_filled, current_texture_unfilled)
# initial_face = render(texture=current_texture, background=background, pixel_uvs=pixel_uvs, background_mask=background_mask)
# save_images(cfg=cfg, _images=initial_face, image_type="face", iteration=-1)

def get_score_fn(sde, score_model):
    def score_fn(x, vec_t):
        labels = sde.marginal_prob(torch.zeros_like(x), vec_t)[1]
        score = score_model(x, labels)
        return score
    return score_fn

probability_flow = False
score_fn = get_score_fn(sde, score_model)
rsde = sde.reverse(score_fn=score_fn, probability_flow=probability_flow)
timesteps = torch.linspace(sde.T, sampling_eps, sde.N)
zeros = torch.zeros(size=image_shape, device=cfg["device"])

with torch.no_grad():
    for i in tqdm(list(range(sde.N))):
        t = timesteps[i]
        vec_t = torch.ones(cfg["batch_size"], device=cfg["device"]) * t
        alpha = torch.ones_like(vec_t)
        
        _, std = sde.marginal_prob(zeros, vec_t)
        std = std * 5 / 34.8
        background = torch.randn(size=image_shape, device=cfg["device"]) * std[:, None, None, None]

        random_tensor_for_texture = torch.randn(size=texture_shape, device=cfg["device"]) * std[:, None, None, None]

        if i == 0:
            print(std.mean())
            current_texture_unfilled = random_tensor_for_texture

        if i <= sde.N - cfg["num_optimization_steps_for_filled_part"]:
            # current_texture_filled = initial_texture_filled + torch.randn(size=texture_shape, device=cfg["device"]) * std[:, None, None, None]
            current_texture_filled = random_tensor_for_texture
        
        current_texture = torch.where(filled_mask, current_texture_filled, current_texture_unfilled)

        # correct
        for j in range(cfg["num_corrector_steps"]):
            face = render(texture=current_texture, background=background, pixel_uvs=pixel_uvs, background_mask=background_mask)

            if i % 10 == 0 and j == 0:
                save_images(cfg=cfg, _images=face, image_type="face", iteration=i)
                save_images(cfg=cfg, _images=current_texture, image_type="texture", iteration=i)
                save_images(cfg=cfg, _images=background, image_type="background", iteration=i)

            grad_face = score_fn(face, vec_t)
            render_func = functools.partial(render, background=background, pixel_uvs=pixel_uvs, background_mask=background_mask)
            grad_texture = get_grad_texture(texture=current_texture, grad_face=grad_face, render_func=render_func)
            grad_texture_norm = torch.norm(grad_texture.reshape(grad_texture.shape[0], -1), dim=-1).mean()
            grad_texture_filled = grad_texture * (1 * filled_mask)
            grad_texture_unfilled = grad_texture * (1 - 1 * filled_mask)

            noise_texture = torch.randn_like(current_texture)
            noise_texture_norm = torch.norm(noise_texture.reshape(noise_texture.shape[0], -1), dim=-1).mean()
            step_size_texture = (cfg["snr"] * noise_texture_norm / grad_texture_norm) ** 2 * 2 * alpha

            if j == cfg["num_corrector_steps"] - 1:
                grad_texture_unfilled_sum = grad_texture_unfilled.sum(axis=1)
                grad_texture_unfilled_zero_one = 1 * grad_texture_unfilled_sum.bool().unsqueeze(0).repeat(repeats=[1, 3, 1, 1])

                grad_texture_filled_sum = grad_texture_filled.sum(axis=1)
                grad_texture_filled_zero_one = 1 * grad_texture_filled_sum.bool().unsqueeze(0).repeat(repeats=[1, 3, 1, 1])

            current_texture_unfilled_mean = current_texture_unfilled + step_size_texture[:, None, None, None] * grad_texture_unfilled
            current_texture_unfilled = current_texture_unfilled_mean + torch.sqrt(step_size_texture * 2)[:, None, None, None] * noise_texture * cfg["langevin_noise_coefficient"]

            if i >= sde.N - cfg["num_optimization_steps_for_filled_part"]:
                current_texture_filled_mean = current_texture_filled + step_size_texture[:, None, None, None] * grad_texture_filled
                current_texture_filled = current_texture_filled_mean + torch.sqrt(step_size_texture * 2)[:, None, None, None] * noise_texture * cfg["langevin_noise_coefficient"]

        if i == 0:
            save_images(cfg, _images=grad_texture_unfilled_zero_one, image_type="texture_unfilled_gradient", iteration=i)
            save_images(cfg, _images=(1 * background_mask), image_type="background_mask", iteration=i)

        if i == sde.N - 1:
            save_gif(cfg=cfg)



/local/home/aarslan/miniconda3/envs/score-face/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-07-14 15:43:09.207183: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/local/home/aarslan/miniconda3/envs/score-face/lib/python3.9/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/local/home/aarslan/miniconda3/envs/score-face/lib/python3.9/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt
/local/home/aarslan/miniconda3/envs/score-face/lib/python3.9/site-packages/pytorch3d/io/utils.py:66: Use

tensor(50., device='cuda:0')


100%|██████████| 1000/1000 [14:21<00:00,  1.16it/s]


In [1]:
import sys
sys.path.append("../src")
import itertools
from renderer import Renderer
from utils import get_experiment_name, save_cfg, set_seeds
from model_utils import get_grad_texture, get_grad_texture_and_background
from visualization import save_images, save_gif
import torch
import numpy as np
import functools
import cv2
from tqdm import tqdm
from score_sde.losses import get_optimizer
from score_sde.models.ema import ExponentialMovingAverage
from score_sde.utils import restore_checkpoint
from score_sde.models import utils as mutils
from score_sde.models import ncsnpp
from score_sde.sde_lib import VESDE
from score_sde.configs.ve import ffhq_256_ncsnpp_continuous as configs

cfg = {
    "seed": 0,
    "batch_size": 1,
    "image_size": 256,
    "texture_size": 2048,
    "saved_image_size": 256,
    "num_channels": 3,
    "obj_path": "../assets/40044.obj",
    
    "image_save_dir": "../../results",
    "device": torch.device("cuda:0"),
    "num_corrector_steps": 6,
    "sde_N": 1000,

    "texture_snr": 0.015,
    "texture_initial_noise_coefficient": 50.0,
    "texture_langevin_noise_coefficient": 1,

    "background_snr": 0.075,
    "background_initial_noise_coefficient": 348.0,
    "background_langevin_noise_coefficient": 1,

    "elev": 0.0,
    "azimuth": 0.0,

    "background": "conditional", # fixed, optimize, conditional
    "texture": "ground_truth", # full_noise, slightly_noise, ground_truth,
    "predict": False # True, False
}

cfg["experiment_name"] = get_experiment_name()
set_seeds(cfg)
save_cfg(cfg)

ckpt_filename = "../assets/checkpoint_48.pth"
config = configs.get_config()
sde = VESDE(sigma_min=config.model.sigma_min, sigma_max=config.model.sigma_max, N=cfg["sde_N"])
sampling_eps = 1e-5

batch_size = cfg["batch_size"]
config.training.batch_size = batch_size
config.eval.batch_size = batch_size
config.device = cfg["device"]

sigmas = mutils.get_sigmas(config)
score_model = mutils.create_model(config)
score_model.eval()

optimizer = get_optimizer(config, score_model.parameters())
ema = ExponentialMovingAverage(score_model.parameters(),
                               decay=config.model.ema_rate)
state = dict(step=0, optimizer=optimizer,
             model=score_model, ema=ema)

state = restore_checkpoint(ckpt_filename, state, config.device)
ema.copy_to(score_model.parameters())

def render(texture, background, pixel_uvs, background_mask):
    texture_sampled = torch.nn.functional.grid_sample(torch.flip(texture, dims=[-2]), pixel_uvs, mode='bilinear', padding_mode='border', align_corners=False)
    return torch.where(background_mask, background, texture_sampled)

image_shape = (cfg["batch_size"], cfg["num_channels"], cfg["image_size"], cfg["image_size"])
texture_shape = (cfg["batch_size"], cfg["num_channels"], cfg["texture_size"], cfg["texture_size"])

def get_score_fn(sde, score_model):
    def score_fn(x, vec_t):
        labels = sde.marginal_prob(torch.zeros_like(x), vec_t)[1]
        score = score_model(x, labels)
        return score
    return score_fn

probability_flow = False
score_fn = get_score_fn(sde, score_model)
rsde = sde.reverse(score_fn=score_fn, probability_flow=probability_flow)
timesteps = torch.linspace(sde.T, sampling_eps, sde.N)
zeros = torch.zeros(size=image_shape, device=cfg["device"])

if cfg["background"] in ["conditional", "optimize"]:
    background = torch.randn(size=image_shape, device=cfg["device"]) * cfg["background_initial_noise_coefficient"]
else:
    background = torch.zeros(size=image_shape, device=cfg["device"])

if cfg["texture"] == "full_noise":
    texture = torch.randn(size=texture_shape, device=cfg["device"]) * cfg["texture_initial_noise_coefficient"]
elif cfg["texture"] == "slightly_noise":
    texture = torch.tensor(cv2.imread("../assets/texture.png")[:, :, [2, 1, 0]], device=cfg["device"]).permute(2, 0, 1).unsqueeze(0) / 255.0 + 0.75 * torch.randn(size=texture_shape, device=cfg["device"])
elif cfg["texture"] == "ground_truth":
    

prerender_results = {}
elevs = [0.0]
azimuths = [0.0]
renderer = Renderer(cfg=cfg)

for elev, azimuth in tqdm(list(itertools.product(elevs, azimuths))):
    pixel_uvs, background_mask = renderer.render(texture=texture, background=background, elev=elev, azimuth=azimuth, result_keys=["pixel_uvs", "background_mask"])
    prerender_results[f"{elev}_{azimuth}"] = {"pixel_uvs": pixel_uvs,
                                              "background_mask": background_mask}

pixel_uvs = torch.tensor(prerender_results["0.0_0.0"]["pixel_uvs"], device=cfg["device"])
background_mask = torch.tensor(prerender_results["0.0_0.0"]["background_mask"], device=cfg["device"]).bool()

face = render(texture=texture, background=background, pixel_uvs=pixel_uvs, background_mask=background_mask)

with torch.no_grad():
    for i in tqdm(list(range(sde.N))):
        if i == 1:
            save_images(cfg, _images=grad_texture_zero_one, image_type="texture_gradient", iteration=i)
            save_images(cfg, _images=(1 * background_mask), image_type="background_mask", iteration=i)
        if (i + 1) % 10 == 0:
            save_images(cfg=cfg, _images=face, image_type="face", iteration=i)
            save_images(cfg=cfg, _images=texture, image_type="texture", iteration=i)
            save_images(cfg=cfg, _images=background, image_type="background", iteration=i)
        if i == sde.N - 1:
            save_gif(cfg=cfg)

        t = timesteps[i]
        vec_t = torch.ones(cfg["batch_size"], device=cfg["device"]) * t
        alpha = torch.ones_like(vec_t)
        
        if cfg["background"] == "conditional":
            background_mean, std = sde.marginal_prob(zeros, vec_t)
            background = background_mean + torch.randn(size=image_shape, device=cfg["device"]) * std[:, None, None, None]

        if cfg["predict"]:
            f, G = rsde.discretize(face, vec_t)
            z = torch.randn_like(face)
            face_mean = face - f
            face = face_mean + G[:, None, None, None] * z

            if cfg["background"] == "optimize":
                background_mean = background - f
                background = background_mean + G[:, None, None, None] * z
            elif cfg["background"] in ["conditional", "fixed"]:
                pass

        # correct
        for j in range(cfg["num_corrector_steps"]):
            face = render(texture=texture, background=background, pixel_uvs=pixel_uvs, background_mask=background_mask)
            grad_face = score_fn(face, vec_t)
            grad_face_norm = torch.norm(grad_face.reshape(grad_face.shape[0], -1), dim=-1).mean()
            if cfg["background"] == "optimize":
                render_func = functools.partial(render, pixel_uvs=pixel_uvs, background_mask=background_mask)
                grad_texture, grad_background = get_grad_texture_and_background(texture=texture, background=background, grad_face=grad_face, render_func=render_func)
            else:
                render_func = functools.partial(render, background=background, pixel_uvs=pixel_uvs, background_mask=background_mask)
                grad_texture = get_grad_texture(texture=texture, grad_face=grad_face, render_func=render_func)

            if j == cfg["num_corrector_steps"] - 1:
                grad_texture_sum = grad_texture.sum(axis=1)
                grad_texture_zero_one = 1 * torch.logical_and(grad_texture_sum, torch.ones_like(grad_texture_sum)).unsqueeze(0).repeat(repeats=[1, 3, 1, 1])
                noise_texture = torch.randn_like(texture)
                noise_texture_norm = torch.norm(noise_texture.reshape(noise_texture.shape[0], -1), dim=-1).mean()
                grad_texture_norm = torch.norm(grad_texture.reshape(grad_texture.shape[0], -1), dim=-1).mean()
                step_size_texture = (cfg["texture_snr"] * noise_texture_norm / grad_texture_norm) ** 2 * 2 * alpha
                texture_mean = texture + step_size_texture[:, None, None, None] * grad_texture
                texture = texture_mean + torch.sqrt(step_size_texture * 2)[:, None, None, None] * noise_texture * cfg["texture_langevin_noise_coefficient"]
            
            if cfg["background"] == "optimize":
                noise_background = torch.randn_like(background)
                noise_background_norm = torch.norm(noise_background.reshape(noise_background.shape[0], -1), dim=-1).mean()
                grad_background_norm = torch.norm(grad_background.reshape(grad_background.shape[0], -1), dim=-1).mean()
                step_size_background = (cfg["background_snr"] * noise_background_norm / grad_background_norm) ** 2 * 2 * alpha
                background = background + step_size_background[:, None, None, None] * grad_background
                background = background + torch.sqrt(step_size_background * 2)[:, None, None, None] * noise_background * cfg["background_langevin_noise_coefficient"]


/local/home/aarslan/miniconda3/envs/score-face/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-07-12 17:41:02.368124: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/local/home/aarslan/miniconda3/envs/score-face/lib/python3.9/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/local/home/aarslan/miniconda3/envs/score-face/lib/python3.9/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt
/local/home/aarslan/miniconda3/envs/score-face/lib/python3.9/site-packages/pytorch3d/io/utils.py:66: Use

KeyboardInterrupt: 